# Keyword graph Retrieve test

In [ ]:
import json
from GraphDB.LegalGraphDB import LegalGraphDB

CONFIG_PATH = "C:/Users/Shic/legal_graph/codes/configs/config.json"

# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")

dbms = LegalGraphDB(auradb=False, config=config)

## Extract User Query Keyword

In [1]:
import json, os

from openai import OpenAI
from dotenv import load_dotenv

CONFIG_PATH = "C:/Users/Shic/legal_graph/codes/configs/config.json"


# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")


# 프롬프트 불러오기
with open("../codes/GraphDB/prompt/query_keyword_extractor.txt", 'r', encoding='utf-8') as file:
    USER_KEYWORD_EXTRACT_PROMPT = file.read()

load_dotenv(verbose=True)
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

Loaded config data from C:/Users/Shic/legal_graph/codes/configs/config.json


In [2]:
def extract_user_keyword(query):
    completion = client.chat.completions.create(
            model=config['model'],
            messages=[
                {"role": "system", "content": USER_KEYWORD_EXTRACT_PROMPT},
                {
                    "role": "user",
                    "content": query
                }
            ]
    )

    print(f"## Query : {query}/n/nKeyword : {completion.choices[0].message.content}")

    answer = completion.choices[0].message.content
    answer = [x.strip() for x in answer.split("|")]
    return answer

In [3]:
QUERY_1 = "증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?"
# QUERY_2 = "회사채의 무보증 후순위사채 기업실사(Due Diligence)를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?"
# QUERY_3 = "증권신고서의 효력 발생기간은?"
# QUERY_4 = "증권신고서 발행을 위해서는 어떤 첨부서류가 필요한가?"
# QUERY_5 = "정정신고가 증권신고서에 어떤 효력을 미치는가?"
# QUERY_6 = "증권신고서와 투자설명서의 부실기재에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?"
# QUERY_7 = "정정신고서를 부실기재했을 경우 과징금은?"
# QUERY_8 = "증권신고서 부실기재 과징금은?"

In [4]:
ANSWER_1 = extract_user_keyword(QUERY_1)
# ANSWER_2 = extract_user_keyword(QUERY_2)
# ANSWER_3 = extract_user_keyword(QUERY_3)
# ANSWER_4 = extract_user_keyword(QUERY_4)
# ANSWER_5 = extract_user_keyword(QUERY_5)
# ANSWER_6 = extract_user_keyword(QUERY_6)
# ANSWER_7 = extract_user_keyword(QUERY_7)
# ANSWER_8 = extract_user_keyword(QUERY_8)

## Query : 증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?/n/nKeyword : 증권신고서 | 정정신고서 | 거짓 기재 | 중요사항 | 배상책임


## Retrieve From Keyword Graph

1. type이 'keyword' 인 node 중에서, 해당하는 name을 가진 node와 연관된 subgraph를 모두 가져온다
2. triplet 형태로 변환

In [5]:
from GraphDB.LegalGraphDB import LegalGraphDB

# db 불러오기
dbms = LegalGraphDB(auradb=False, config=config)

## Neo4j AuraDB 서버와의 연결이 성공적으로 확인되었습니다.


In [6]:
def get_keyword_subgraphs(keyword_list):
    triplets = []
    triplet_ids = set()  # To ensure uniqueness

    with dbms.driver.session(database="legal-graph") as session:
        for keyword_name in keyword_list:
                query = """
                    MATCH (n:keyword {name: $name})-[r]-(m)
                    RETURN n, r, m
                    """
                subgraph_result = session.run(query, name=keyword_name)

                for record in subgraph_result:
                    n = record["n"]
                    r = record["r"]
                    m = record["m"]
                    # Use IDs to ensure uniqueness since Node and Relationship objects are not hashable
                    triplet_id = (n.element_id, r.element_id, m.element_id)
                    if triplet_id not in triplet_ids:
                        triplet_ids.add(triplet_id)
                        triplets.append((n, r, m))
    
    return list(set(triplets))

# 전체 반환
def all_triplet(query, answer):
    answer_triplets = get_keyword_subgraphs(answer)
    important_triplets = []
    # Output the results
    print('## User query : ', query)
    print('Keyword : ', answer)
    # print()
    # print("Triplets:")
    for n, r, m in answer_triplets:
        # print(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
        important_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
    
    return list(set(important_triplets))


def exact_match_triplet(query, answer):
    answer_triplets = get_keyword_subgraphs(answer)
    important_triplets = []
    # Output the results
    print('## User query : ', query)
    print('Keyword : ', answer)
    # print()
    # print("Triplets:")
    for n, r, m in answer_triplets:
        # print(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
        if m['name'] in answer:
            important_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
    
    return list(set(important_triplets))

def partial_match_triplet(query, answer):
    answer_triplets = get_keyword_subgraphs(answer)
    important_triplets = []
    # Output the results
    print('## User query : ', query)
    print('Keyword : ', answer)
    # print()
    # print("Triplets:")
    for n, r, m in answer_triplets:
        # print(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
         # 부분 일치를 확인하여 중요한 triplet에 포함
        if any(keyword in m['name'] for keyword in answer):
            important_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
    
    return list(set(important_triplets))

In [7]:
exact_1_triplets = exact_match_triplet(QUERY_1, ANSWER_1)
exact_1_triplets

## User query :  증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?
Keyword :  ['증권신고서', '정정신고서', '거짓 기재', '중요사항', '배상책임']


['(정정신고서 -[INCLUDED_IN]-> 증권신고서) IN 시행령_제68조제5항_불건전 영업행위의 금지',
 '(증권신고서 -[INCLUDED_IN]-> 정정신고서) IN 시행령_제68조제5항_불건전 영업행위의 금지']

In [ ]:
partial_1_triplets = partial_match_triplet(QUERY_1, ANSWER_1)
partial_1_triplets

In [ ]:
exact_2_triplets = exact_match_triplet(QUERY_2, ANSWER_2)
exact_2_triplets

In [ ]:
partial_2_triplets = partial_match_triplet(QUERY_2, ANSWER_2)
partial_2_triplets

In [ ]:
exact_3_triplets = exact_match_triplet(QUERY_3, ANSWER_3)
exact_3_triplets

In [ ]:
partial_3_triplets = partial_match_triplet(QUERY_3, ANSWER_3)
partial_3_triplets

In [ ]:
exact_4_triplets = exact_match_triplet(QUERY_4, ANSWER_4)
exact_4_triplets

In [ ]:
partial_4_triplets = partial_match_triplet(QUERY_4, ANSWER_4)
partial_4_triplets

In [ ]:
exact_5_triplets = exact_match_triplet(QUERY_5, ANSWER_5)
exact_5_triplets

In [ ]:
partial_5_triplets = partial_match_triplet(QUERY_5, ANSWER_5)
partial_5_triplets

In [ ]:
exact_6_triplets = exact_match_triplet(QUERY_6, ANSWER_6)
exact_6_triplets

In [ ]:
partial_6_triplets = partial_match_triplet(QUERY_6, ANSWER_6)
partial_6_triplets

In [ ]:
exact_7_triplets = exact_match_triplet(QUERY_7, ANSWER_7)
exact_7_triplets

In [ ]:
partial_7_triplets = partial_match_triplet(QUERY_7, ANSWER_7)
partial_7_triplets

In [ ]:
exact_8_triplets = exact_match_triplet(QUERY_8, ANSWER_8)
exact_8_triplets

In [ ]:
partial_8_triplets = partial_match_triplet(QUERY_8, ANSWER_8)
partial_8_triplets

### 기타 법률 전문가 QA 목록

> 하나도 안나옴 ...... 데이터 범위 밖인듯

In [5]:
QUERY_9 = "기업실사의 근거규정은?"
QUERY_10 = "주주배정 후 실권주 일반공모 우상증자 인수업무를 하는 경우에도 거래제한대상 종목등록을 해야하는지?"
QUERY_11 = "주권의 모집주선, 인수계약에 따른 조사분석제한/이해관계고지대상법인관리의 경우 제한기간은?"
QUERY_12 = "자산유동화법상의 유동화증권 주관회사 및 업무수탁인인 경우, 자산유동화에 관한 법률 제33조의 3 단서에 따라 본건 유동화증권이 의무보유 면제 대상인지?"

In [ ]:
ANSWER_9 = extract_user_keyword(QUERY_9)
ANSWER_10 = extract_user_keyword(QUERY_10)
ANSWER_11 = extract_user_keyword(QUERY_11)
ANSWER_12 = extract_user_keyword(QUERY_12)

In [ ]:
exact_9_triplets = exact_match_triplet(QUERY_9, ANSWER_9)
exact_9_triplets

In [ ]:
partial_9_triplets = partial_match_triplet(QUERY_9, ANSWER_9)
partial_9_triplets

In [15]:
for keyword in ANSWER_9:
    get_subgraphs([keyword])

In [ ]:
exact_10_triplets = exact_match_triplet(QUERY_10, ANSWER_10)
exact_10_triplets

In [ ]:
partial_10_triplets = partial_match_triplet(QUERY_10, ANSWER_10)
partial_10_triplets

In [16]:
for keyword in ANSWER_10:
    get_subgraphs([keyword])

In [19]:
for keyword in ANSWER_12:
    get_subgraphs([keyword])

In [ ]:
exact_11_triplets = exact_match_triplet(QUERY_11, ANSWER_11)
exact_11_triplets

In [ ]:
partial_11_triplets = partial_match_triplet(QUERY_11, ANSWER_11)
partial_11_triplets

## Generate Answer by retrieved triplet keyword


In [ ]:
from GraphDB.LegalGraphDB import LegalGraphDB

import json, os

from openai import OpenAI
from dotenv import load_dotenv

CONFIG_PATH = "C:/Users/Shic/legal_graph/codes/configs/config.json"


# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")


# 프롬프트 불러오기
with open("../codes/GraphDB/prompt/query_answer.txt", 'r', encoding='utf-8') as file:
    GENERATE_ANSWER_PROMPT = file.read()
with open("../codes/GraphDB/prompt/query_keyword_extractor.txt", 'r', encoding='utf-8') as file:
    USER_KEYWORD_EXTRACT_PROMPT = file.read()


load_dotenv(verbose=True)
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# db 불러오기
dbms = LegalGraphDB(auradb=False, config=config)

# 프롬프트 불러오기


def extract_user_keyword(query):
    completion = client.chat.completions.create(
            model=config['model'],
            messages=[
                {"role": "system", "content": USER_KEYWORD_EXTRACT_PROMPT},
                {
                    "role": "user",
                    "content": query
                }
            ]
    )

    print(f"## Query : {query}/n/nKeyword : {completion.choices[0].message.content}")

    answer = completion.choices[0].message.content
    answer = [x.strip() for x in answer.split("|")]
    return answer


def get_subgraphs(keyword_list):
    triplets = []
    triplet_ids = set()  # To ensure uniqueness

    with dbms.driver.session(database="legal-graph") as session:
        for keyword_name in keyword_list:
                query = """
                    MATCH (n:keyword {name: $name})-[r]-(m)
                    RETURN n, r, m
                    """
                subgraph_result = session.run(query, name=keyword_name)

                for record in subgraph_result:
                    n = record["n"]
                    r = record["r"]
                    m = record["m"]
                    # Use IDs to ensure uniqueness since Node and Relationship objects are not hashable
                    triplet_id = (n.element_id, r.element_id, m.element_id)
                    if triplet_id not in triplet_ids:
                        triplet_ids.add(triplet_id)
                        triplets.append((n, r, m))
    
    return list(set(triplets))

def exact_match_triplet(query, answer):
    answer_triplets = get_subgraphs(answer)

    total_triplets = []
    important_triplets = []
    # Output the results
    print('## User query : ', query)
    print('Keyword : ', answer)
    # print()
    print("## Triplets : ", len(answer_triplets))
    for n, r, m in answer_triplets:
        total_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
        print('## m entity name : ', m['name'])
        if m['name'] in answer:
            important_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
    
    return list(set(total_triplets)), list(set(important_triplets))

def partial_match_triplet(query, answer):
    answer_triplets = get_subgraphs(answer)

    total_triplets = []
    important_triplets = []
    # Output the results
    print('## User query : ', query)
    print('Keyword : ', answer)
    # print()
    # print("Triplets:")
    for n, r, m in answer_triplets:
        total_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
         # 부분 일치를 확인하여 중요한 triplet에 포함
        if any(keyword in m['name'] for keyword in answer):
            important_triplets.append(f"({n['name']} -[{r.type}]-> {m['name']}) IN {r['reference']}")
    
    return list(set(total_triplets)), list(set(important_triplets))

### Query 6 test

In [ ]:
QUERY_6 = "투자설명서의 부실기재에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?"
ANSWER_6 = extract_user_keyword(QUERY_6)
ANSWER_6

In [ ]:
total_6_triplets, exact_6_triplets = exact_match_triplet(QUERY_6, ANSWER_6)
total_6_triplets

In [4]:
query = f"""**User Query**
{QUERY_6}

**keyword triplet**
{total_6_triplets}

[output]
"""

In [ ]:
query

In [6]:
def generated_answer_based_on_keyword(query):
    completion = client.chat.completions.create(
            model=config['model'],
            messages=[
                {"role": "system", "content": GENERATE_ANSWER_PROMPT},
                {
                    "role": "user",
                    "content": query
                }
            ]
    )

    answer = completion.choices[0].message.content
    return answer

In [ ]:
generated_answer_based_on_keyword(query)

### Query 8

In [ ]:
QUERY_8 = "증권신고서 부실기재 과징금은?"
ANSWER_8 = extract_user_keyword(QUERY_8)
ANSWER_8

In [ ]:
total_8_triplets, exact_8_triplets = exact_match_triplet(QUERY_8, ANSWER_8)
total_8_triplets

In [10]:
query = f"""**User Query**
{QUERY_8}

**keyword triplet**
{total_8_triplets}

[output]
"""

In [ ]:
query

In [ ]:
generated_answer_based_on_keyword(query)

### 라이브러리 이용

- 수동으로 cypher query 입력해서 반환해오는 게 나은지
- neo4j llamaindex / langchain 라이브러리 내 구현되어 있는 cypher query 생성 LLM을 활용하는 것이 나은지

-> 계속 cypher query schema error 뜸...
LLM이 cypher query를 생성하다보니 자꾸 없는 node, relationship 만들어냄
-> schema 제약을 줘도 동일한 에러가 계속 발생해서, 우선은 위의 수동 프롬프트 방식대로 진행하는 게 나을듯


#### llamaindex - Neo4jQueryToolSpec

- QueryToolSpec (https://llamahub.ai/l/tools/llama-index-tools-neo4j?from=) 사용

In [ ]:
from llama_index.tools.neo4j import Neo4jQueryToolSpec
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent

# Define a custom schema extraction prompt to include necessary nodes and relationships
custom_schema_extract_prompt = """
Please extract the schema of the Neo4j database, including all nodes and relationships relevant to penalty subjects and penalty amounts.
The schema should include:
- Nodes: keyword
- Relationships: SAME_AS, INCLUDED_IN 
"""

gds_db = Neo4jQueryToolSpec(
    url=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    llm=OpenAI(temperature=0, model="gpt-4o"),
    database="legal-graph",
)

tools = gds_db.to_tool_list()
agent = OpenAIAgent.from_tools(tools, verbose=True)

agent.chat(QUERY_6)

cypher query extract 수정

#### langchain - GraphCypherQAChain

In [1]:
import os
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain_community.graphs import Neo4jGraph

enhanced_graph = Neo4jGraph(url = os.getenv("NEO4J_URI"),
                            username = os.getenv("NEO4J_USERNAME"),
                            password = os.getenv("NEO4J_PASSWORD"),
                            enhanced_schema=True)
enhanced_graph.schema

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


'Node properties:\n\nRelationship properties:\n\nThe relationships:\n'

In [ ]:
# chain = GraphCypherQAChain.from_llm(
#     ChatOpenAI(temperature=0, model="gpt-4"), graph=enhanced_graph, verbose=True,
# )
# chain.run(QUERY_6)

In [7]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

CYPHER_GENERATION_KOREAN_TEMPLATE = """지시사항을 따라 GraphDB를 탐색 및 조회하기 위한 Cypher Query를 생성하세요.
<지시사항>
- 주어진 스키마의 관계 유형과 속성만 사용하세요.
    <스키마>
    {schema}
    </스키마>
- 제공되지 않은 type이나 property는 사용하지 마세요.
- Cypher 문을 생성하는 것 이외의 질문에는 응답하지 마세요.
- 생성된 Cypher 문 이외의 텍스트는 포함하지 마세요.
</지시사항>

다음은 특정 질문에 대해 생성된 Cypher Query 예시입니다.
<예시>
input: 정정신고서에 거짓기재가 포함되어 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?
MATCH (n:keyword)
WHERE name IN ['정정신고서', '거짓기재', '투자자', '손해', '배상책임'] WHERE n.name CONTAINS name
RETURN n
LIMIT 5
</예시>

input: {question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_KOREAN_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    graph=enhanced_graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    use_function_response=True,
    function_response_system="당신은 자본시장법을 잘 알고 있는 법률 전문가입니다. 반환된 문서 내용을 기반으로 친절하고 자세하게 답변해주세요.",
)


c:\Users\Shic\.conda\envs\neo4j\Lib\site-packages\pydantic\v1\main.py:719: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  return cls(**value_as_dict)


In [8]:
chain.invoke({"query": "정정신고서가 증권신고서에 어떤 효력을 미치나요?"})



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: keyword)} {position: line: 1, column: 10, offset: 9} for query: "MATCH (n:keyword)\nWHERE n.name IN ['정정신고서', '증권신고서', '효력']\nRETURN n\nLIMIT 5"


Generated Cypher:
MATCH (n:keyword)
WHERE n.name IN ['정정신고서', '증권신고서', '효력']
RETURN n
LIMIT 5
Full Context:
[]

> Finished chain.


{'query': '정정신고서가 증권신고서에 어떤 효력을 미치나요?',
 'result': '정정신고서는 증권신고서의 내용을 수정하거나 보완하기 위해 제출되는 문서입니다. 정정신고서가 제출되면, 그 내용은 원래의 증권신고서에 포함된 것으로 간주됩니다. 이는 투자자들이 최신의 정확한 정보를 바탕으로 투자 결정을 내릴 수 있도록 하기 위함입니다. 따라서 정정신고서가 제출되면, 그 내용은 증권신고서의 일부로서 법적 효력을 가지며, 투자자들에게 제공되는 모든 정보는 정정된 내용을 반영해야 합니다. \n\n또한, 정정신고서 제출로 인해 증권신고서의 효력 발생일이 변경될 수 있으며, 이는 투자자 보호와 관련된 중요한 절차입니다. 정정신고서가 제출된 경우, 투자자들은 정정된 내용을 충분히 검토할 수 있는 시간을 가질 수 있도록 일정 기간 동안 증권의 발행이 연기될 수 있습니다.'}